### DATA COLLECTION

In [ ]:
# Necessary Imports
from flask import Flask, jsonify
from werkzeug.routing import IntegerConverter
from _csv import Dialect as _Dialect
import copy
import csv
import os
from lib.rtls_slave import pixel_calculate, aoa_main, sort_data
import json


ENVIRONMENT = 'random'

# Handle Negative Values
class SignedIntConverter(IntegerConverter):
    regex = r'-?\d+'
app = Flask(__name__)
app.url_map.converters['signed_int'] = SignedIntConverter
#Store input values: AoA and Horizontal Angle

aoabefore = {}
aoa= {}
lev = []

@app.route('/')
def home():
    return "This is the HomePage"

#Routing Settings
@app.route('/send/<signed_int:yaw>/<signed_int:roll>/<signed_int:pitch>', methods=['GET'])
def get_angle(yaw,roll,pitch):
    print(f"YAW: {yaw}, ROLL: {roll}, PITCH: {pitch}")
    global aoa
    global aoabefore
    
    print("Plane Angle: " + str(roll))
    aoa_result = aoa_main()
    input_letter = input("If you want to add this data to list type 'y', otherwise press Enter")
    if input_letter!='y':
        return jsonify({'state':'Try Again'})
    lev.append(roll)
    aoabefore = copy.deepcopy(aoa)
    aoa = aoa_result

    with open(f"./data/aoa_data/{ENVIRONMENT}_{roll}.json",'w') as f:
        json.dump(aoa_result, f)
        
    return jsonify({'state': 'success'})

@app.route('/calculate/', methods=['GET'])
def calculate():
    # ANGLE SORTING TECHNIQUES
    aoabefore_ = sort_data(aoabefore,aoa_bias = -35)
    aoa_ = sort_data(aoa,aoa_bias = -35)

    print("Before Turning："+str(aoabefore)+'\n'+"After Turning： " + str(aoa) +'\n'+"Rotation Angle： "+ str(lev))
    set_aoa = set(aoa_)
    set_aoabefore = set(aoabefore_)

    for slave in set_aoa.intersection(set_aoabefore):
        f = open(f"./results/pixels/{slave.replace(':','_')}.csv",'w',newline="")
        f.truncate()
        writer = csv.writer(f)
        
        data_before = aoabefore_[slave]
        data_after = aoa_[slave]

        for i in range(len(data_before[0])):
            for j in range(len(data_after[0])):
                (temp1, temp2) = pixel_calculate(lev[-2] - lev[-1], data_before[0][i], data_after[0][j])
                count = data_before[1][i]*data_after[1][j]
                if temp1 != -1:
                    tux = (temp1, temp2,count)
                    writer.writerow(tux)

        f.close()
                
        
        f.close()

    
    return jsonify({'state': 'success'})

In [ ]:
ENVIRONMENT = 'random'
app.run(host='0.0.0.0',port = 5000, debug=False)
# My Charger: 80:6F:B0:EE:AD:32 : Water Bottle.
# Ishrath Charger:  80:6F:B0:EE:AA:13 : Tea Bottle.

### POST POINT CALCULATION

In [4]:
from lib.rtls_slave import post_calculation

count = 0
with open("./data/weights/calculation_reference.txt", "r") as file:
    for line in file:
        count = count + 1
        ctnt = line.split('_')
        CASE = '_'.join(ctnt[:-2])
        L1 = int(ctnt[-2])
        L2 = int(ctnt[-1])
        print(f"CASE {count}: {CASE}")
        post_calculation(CASE,L1,L2,aoa_bias = -45)

CASE 1: calib1
Rotation Angle: 68 --> 74

Slave :80:6F:B0:EE:AA:13 
	AoA Before:[-7.         -6.33333333 -5.66666667 -5.        ] 
	AoA After:[-7.  -6.4 -5.8 -5.2 -4.6 -4. ]
	Center: (2465.6826567711387, 1804.2091101718602)

CASE 2: calib2
Rotation Angle: 68 --> 74

Slave :80:6F:B0:EE:AA:13 
	AoA Before:[-1.         -0.33333333  0.33333333  1.        ] 
	AoA After:[0.         0.66666667 1.33333333 2.        ]
	Center: (2000, 1500)

CASE 3: calib3
Rotation Angle: 68 --> 74

Slave :80:6F:B0:EE:AA:13 
	AoA Before:[33.         33.57142857 34.14285714 34.71428571 35.28571429 35.85714286
 36.42857143] 
	AoA After:[38.  38.6 39.2 39.8 40.4 41. ]
	Center: (-654.6612169872315, -1116.875686774537)

CASE 4: calib4
Rotation Angle: 68 --> 74

Slave :80:6F:B0:EE:AA:13 
	AoA Before:[29.         29.66666667 30.33333333 31.        ] 
	AoA After:[29.         29.57142857 30.14285714 30.71428571 31.28571429 31.85714286
 32.42857143]
	Center: (-1096.3253988348342, 1399.0833459661012)

CASE 5: calib5
Rotati

f:\intern_VisBLE\VisBLE_INTERN\codes\lib\rtls_slave.py:300: RuntimeWarning: invalid value encountered in arccos
  azimuth_angle = np.arccos(np.cos(angle1) / np.sin(elevation_angle))
f:\intern_VisBLE\VisBLE_INTERN\codes\lib\rtls_slave.py:300: RuntimeWarning: invalid value encountered in arccos
  azimuth_angle = np.arccos(np.cos(angle1) / np.sin(elevation_angle))
f:\intern_VisBLE\VisBLE_INTERN\codes\lib\rtls_slave.py:300: RuntimeWarning: invalid value encountered in arccos
  azimuth_angle = np.arccos(np.cos(angle1) / np.sin(elevation_angle))
f:\intern_VisBLE\VisBLE_INTERN\codes\lib\rtls_slave.py:300: RuntimeWarning: invalid value encountered in arccos
  azimuth_angle = np.arccos(np.cos(angle1) / np.sin(elevation_angle))
f:\intern_VisBLE\VisBLE_INTERN\codes\lib\rtls_slave.py:300: RuntimeWarning: invalid value encountered in arccos
  azimuth_angle = np.arccos(np.cos(angle1) / np.sin(elevation_angle))
f:\intern_VisBLE\VisBLE_INTERN\codes\lib\rtls_slave.py:300: RuntimeWarning: invalid value 

Rotation Angle: 64 --> 72

Slave :80:6F:B0:EE:9C:67 
	AoA Before:[-8.  -7.4 -6.8 -6.2 -5.6 -5. ] 
	AoA After:[-8.8  -8.28 -7.76 -7.24 -6.72 -6.2  -5.68 -5.16]
	Center: (2499.8176290345864, 1824.0059365748461)

CASE 31: test20
Rotation Angle: 64 --> 72

Slave :80:6F:B0:EE:9C:67 
	AoA Before:[25.55555556 26.11111111 26.66666667 27.22222222 27.77777778 28.33333333
 28.88888889] 
	AoA After:[20.  20.6 21.2 21.8 22.4 23. ]
	Center: (260.36657265058216, 3628.905237934352)

CASE 32: test21
Rotation Angle: 52 --> 62

Slave :80:6F:B0:EE:9C:67 
	AoA Before:[1.         1.66666667 2.33333333 3.        ] 
	AoA After:[-1.  -0.4  0.2  0.8  1.4  2. ]
	Center: (-1, -1)

CASE 33: test21
Rotation Angle: 57 --> 67

Slave :80:6F:B0:EE:9C:67 
	AoA Before:[1. 2.] 
	AoA After:[2.  2.6 3.2 3.8 4.4 5. ]
	Center: (1572.5609756717474, 1506.5600874672482)

CASE 34: test21
Rotation Angle: 62 --> 72

Slave :80:6F:B0:EE:9C:67 
	AoA Before:[-1.  -0.4  0.2  0.8  1.4  2. ] 
	AoA After:[0.63636364 1.18181818 1.72727273 2

f:\intern_VisBLE\VisBLE_INTERN\codes\lib\rtls_slave.py:300: RuntimeWarning: invalid value encountered in arccos
  azimuth_angle = np.arccos(np.cos(angle1) / np.sin(elevation_angle))
f:\intern_VisBLE\VisBLE_INTERN\codes\lib\rtls_slave.py:300: RuntimeWarning: invalid value encountered in arccos
  azimuth_angle = np.arccos(np.cos(angle1) / np.sin(elevation_angle))
f:\intern_VisBLE\VisBLE_INTERN\codes\lib\rtls_slave.py:300: RuntimeWarning: invalid value encountered in arccos
  azimuth_angle = np.arccos(np.cos(angle1) / np.sin(elevation_angle))
f:\intern_VisBLE\VisBLE_INTERN\codes\lib\rtls_slave.py:300: RuntimeWarning: invalid value encountered in arccos
  azimuth_angle = np.arccos(np.cos(angle1) / np.sin(elevation_angle))
f:\intern_VisBLE\VisBLE_INTERN\codes\lib\rtls_slave.py:300: RuntimeWarning: invalid value encountered in arccos
  azimuth_angle = np.arccos(np.cos(angle1) / np.sin(elevation_angle))
f:\intern_VisBLE\VisBLE_INTERN\codes\lib\rtls_slave.py:300: RuntimeWarning: invalid value 

Rotation Angle: 77 --> 87

Slave :80:6F:B0:EE:9C:67 
	AoA Before:[-1.31578947 -0.78947368 -0.26315789  0.26315789  0.78947368  1.31578947
  1.84210526] 
	AoA After:[1.17948718 1.69230769 2.20512821 2.71794872 3.23076923 3.74358974
 4.25641026 4.76923077]
	Center: (-1, -1)

CASE 70: test28
Rotation Angle: 66 --> 76

Slave :80:6F:B0:EE:9C:67 
	AoA Before:[18.  18.6 19.2 19.8 20.4 21. ] 
	AoA After:[18.  18.6 19.2 19.8 20.4 21. ]
	Center: (169.64262568682284, 1980.6263559906379)

CASE 71: test28
Rotation Angle: 71 --> 81

Slave :80:6F:B0:EE:9C:67 
	AoA Before:[20.  20.6 21.2 21.8 22.4 23. ] 
	AoA After:[18.         18.66666667 19.33333333 20.        ]
	Center: (123.49346569264526, 2435.5372022062597)

CASE 72: test28
Rotation Angle: 76 --> 86

Slave :80:6F:B0:EE:9C:67 
	AoA Before:[18.  18.6 19.2 19.8 20.4 21. ] 
	AoA After:[20.         20.66666667 21.33333333 22.        ]
	Center: (122.3843143677334, 1676.9617819928621)

CASE 73: test28
Rotation Angle: 81 --> 91

Slave :80:6F:B0:EE:9C:67

f:\intern_VisBLE\VisBLE_INTERN\codes\lib\rtls_slave.py:300: RuntimeWarning: invalid value encountered in arccos
  azimuth_angle = np.arccos(np.cos(angle1) / np.sin(elevation_angle))
f:\intern_VisBLE\VisBLE_INTERN\codes\lib\rtls_slave.py:300: RuntimeWarning: invalid value encountered in arccos
  azimuth_angle = np.arccos(np.cos(angle1) / np.sin(elevation_angle))
e:\Python_VENV\AoA\lib\site-packages\sklearn\neighbors\_lof.py:282: UserWarning: n_neighbors (20) is greater than the total number of samples (16). n_neighbors will be set to (n_samples - 1) for estimation.
  warnings.warn(
f:\intern_VisBLE\VisBLE_INTERN\codes\lib\rtls_slave.py:300: RuntimeWarning: invalid value encountered in arccos
  azimuth_angle = np.arccos(np.cos(angle1) / np.sin(elevation_angle))
e:\Python_VENV\AoA\lib\site-packages\sklearn\neighbors\_lof.py:282: UserWarning: n_neighbors (20) is greater than the total number of samples (18). n_neighbors will be set to (n_samples - 1) for estimation.
  warnings.warn(
f:\int

Rotation Angle: 60 --> 70

Slave :80:6F:B0:EE:9C:67 
	AoA Before:[-25.82352941 -25.29411765 -24.76470588 -24.23529412 -23.70588235
 -23.17647059 -22.64705882 -22.11764706] 
	AoA After:[-20.2        -19.68571429 -19.17142857 -18.65714286 -18.14285714
 -17.62857143 -17.11428571 -16.6       ]
	Center: (3704.441855975512, 3158.337821228824)

Slave :80:6F:B0:EE:AD:32 
	AoA Before:[27.6        28.13333333 28.66666667 29.2        29.73333333 30.26666667
 30.8        31.33333333] 
	AoA After:[26.55555556 27.11111111 27.66666667 28.22222222 28.77777778 29.33333333
 29.88888889]
	Center: (-925.0033046843905, 2273.7257974792255)

CASE 99: test46
Rotation Angle: 60 --> 70

Slave :80:6F:B0:EE:9C:67 
	AoA Before:[-25.90909091 -25.36363636 -24.81818182 -24.27272727 -23.72727273
 -23.18181818 -22.63636364 -22.09090909] 
	AoA After:[-27.33333333 -26.8        -26.26666667 -25.73333333 -25.2
 -24.66666667 -24.13333333]
	Center: (4389.48939348558, 1550.5128742832792)

Slave :80:6F:B0:EE:AD:32 
	AoA Before

f:\intern_VisBLE\VisBLE_INTERN\codes\lib\rtls_slave.py:300: RuntimeWarning: invalid value encountered in arccos
  azimuth_angle = np.arccos(np.cos(angle1) / np.sin(elevation_angle))
f:\intern_VisBLE\VisBLE_INTERN\codes\lib\rtls_slave.py:300: RuntimeWarning: invalid value encountered in arccos
  azimuth_angle = np.arccos(np.cos(angle1) / np.sin(elevation_angle))
f:\intern_VisBLE\VisBLE_INTERN\codes\lib\rtls_slave.py:300: RuntimeWarning: invalid value encountered in arccos
  azimuth_angle = np.arccos(np.cos(angle1) / np.sin(elevation_angle))
f:\intern_VisBLE\VisBLE_INTERN\codes\lib\rtls_slave.py:300: RuntimeWarning: invalid value encountered in arccos
  azimuth_angle = np.arccos(np.cos(angle1) / np.sin(elevation_angle))
f:\intern_VisBLE\VisBLE_INTERN\codes\lib\rtls_slave.py:300: RuntimeWarning: invalid value encountered in arccos
  azimuth_angle = np.arccos(np.cos(angle1) / np.sin(elevation_angle))
f:\intern_VisBLE\VisBLE_INTERN\codes\lib\rtls_slave.py:300: RuntimeWarning: invalid value 

CASE 120: test66
Rotation Angle: 65 --> 75

Slave :80:6F:B0:EE:AA:13 
	AoA Before:[-4.42857143 -3.85714286 -3.28571429 -2.71428571 -2.14285714 -1.57142857
 -1.        ] 
	AoA After:[-6.25641026 -5.74358974 -5.23076923 -4.71794872 -4.20512821 -3.69230769
 -3.17948718]
	Center: (2785.7201504402865, 1346.7187418449819)

Slave :80:6F:B0:EE:9C:67 
	AoA Before:[-23.29411765 -22.78431373 -22.2745098  -21.76470588 -21.25490196
 -20.74509804 -20.23529412 -19.7254902 ] 
	AoA After:[-14.76712329 -14.26027397 -13.75342466 -13.24657534 -12.73972603
 -12.23287671 -11.7260274  -11.21917808]
	Center: (2892.1222771695357, 3394.5358508060517)

Slave :80:6F:B0:EE:AD:32 
	AoA Before:[-11.24475524 -10.74125874 -10.23776224  -9.73426573  -9.23076923
  -8.72727273  -8.22377622] 
	AoA After:[57.79136691 58.29496403 58.79856115 59.30215827 59.8057554  60.30935252
 60.81294964 61.31654676]
	Center: (2000, 1500)

CASE 121: test67
Rotation Angle: 60 --> 70

Slave :80:6F:B0:EE:AA:13 
	AoA Before:[-14.38461538 -13.

f:\intern_VisBLE\VisBLE_INTERN\codes\lib\rtls_slave.py:300: RuntimeWarning: invalid value encountered in arccos
  azimuth_angle = np.arccos(np.cos(angle1) / np.sin(elevation_angle))
f:\intern_VisBLE\VisBLE_INTERN\codes\lib\rtls_slave.py:299: RuntimeWarning: invalid value encountered in arccos
  elevation_angle = np.arccos((np.cos(angle2) - np.cos(angle1) * np.cos(level_angle)) / np.sin(level_angle))
f:\intern_VisBLE\VisBLE_INTERN\codes\lib\rtls_slave.py:300: RuntimeWarning: invalid value encountered in arccos
  azimuth_angle = np.arccos(np.cos(angle1) / np.sin(elevation_angle))
f:\intern_VisBLE\VisBLE_INTERN\codes\lib\rtls_slave.py:300: RuntimeWarning: invalid value encountered in arccos
  azimuth_angle = np.arccos(np.cos(angle1) / np.sin(elevation_angle))
f:\intern_VisBLE\VisBLE_INTERN\codes\lib\rtls_slave.py:300: RuntimeWarning: invalid value encountered in arccos
  azimuth_angle = np.arccos(np.cos(angle1) / np.sin(elevation_angle))
f:\intern_VisBLE\VisBLE_INTERN\codes\lib\rtls_slave

### VISUALIZING THE ANGLE

In [ ]:
from lib.rtls_slave import visualize_aoa_spread

CASE = 'three_down_indoor_i'
level = 74
visualize_aoa_spread(CASE, level)

In [ ]:
from lib.rtls_slave import visualize_aoa_turn_spread

CASE = 'two_top_indoor'
level1 = 68
level2 = 76
vis_bias = 0

visualize_aoa_turn_spread(CASE, level1, level2, vis_bias)

### RAW DATA VISUALIZATION

In [ ]:
import matplotlib.pyplot as plt
import json

def visualize_aoa_spread(CASE, level, num_bins = 100):

    with open(f'./data/aoa_data/{CASE}_{level}.json', 'r') as f:
        aoa_data = json.load(f)

    # Create a figure and axis object
    fig, ax = plt.subplots()

    # Set the x and y labels and title
    ax.set_xlabel("Angle of Arrival")
    ax.set_ylabel("# of Occurance")
    ax.set_title("Histogram of Angle of Arrival")

    # Set the number of bins in the histogram

    # Loop over each slave and plot its angle of arrival data as a histogram
    for slave, data in aoa_data.items():
        ax.hist(data, bins=num_bins, alpha=0.5, label=slave)

    # Add a legend to the plot
    ax.legend()

    # Show the plot
    plt.show()

    if len(aoa_data.keys())>1:
        # Set up the figure with subplots
        fig, axs = plt.subplots(nrows=1, ncols=len(aoa_data.keys()), figsize=(10, 5))

        # Iterate over each slave and plot a histogram of its angle of arrival data
        for i, slave in enumerate(aoa_data.keys()):
            axs[i].hist(aoa_data[slave], bins=num_bins,color='lightblue')  # adjust bins as needed
            axs[i].set_title(slave)
            axs[i].set_xlabel("Angle of Arrival")
            axs[i].set_ylabel("# of Occurance")

        # Adjust the spacing between subplots and display the figure
        plt.tight_layout()

        plt.show()
        
        return "SUCESS"
    

visualize_aoa_spread('one_top_indoor',70,100)